In [1]:
%load_ext autoreload

In [2]:
# autoreload chaque module spécifié par %aimport
%autoreload 1

%aimport src.kerasCallbacks
%aimport src.results

from IPython.display import display
from os import path
import time

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from src import constants
from src.kerasCallbacks import BestWeightsRestorer
import src.results as resultsUtil
import src.runUtil as runUtil

In [3]:

def load_feature_set():

    file_path = path.join(constants.DATA_PATH, 'jmirmfccs_base_split.csv')
    
    ftrs = np.array(pd.read_csv(file_path, header=None).values[:,2:-1])
    lbls = np.array(pd.read_csv(file_path, header=None).values[:,-1])
    
    return ftrs, lbls

def scale_features(ftrs):
    
    scaler = StandardScaler()
    ftrs_scld = scaler.fit_transform(ftrs)
    
    return ftrs_scld

def one_hot_labels(lbls):
    
    lbls_1d = lbls.reshape(len(lbls), 1)
        
    oh_encoder = OneHotEncoder(sparse=False)
    lbls_oh = oh_encoder.fit_transform(lbls_1d)
    
    return lbls_oh

In [4]:
# Load the features and the labels
raw_ftrs, raw_lbls = load_feature_set()

lbls_oh = one_hot_labels(raw_lbls)
lbl_encoder = LabelEncoder()
lbl_encoded_lbls = lbl_encoder.fit_transform(raw_lbls)

ftrs = scale_features(raw_ftrs)

print(ftrs.shape)

(143644, 26)


In [8]:
def create_model(input_shape):
    
    inputs = Input(shape=input_shape, name='mlp_mfccs_input')
    
    layer = Dense(26, activation='relu', name='mlp_mfccs_dense_1')(inputs)
    layer = Dense(60, activation='relu', name='mlp_mfccs_dense_2')(layer)
    layer = Dense(45, activation='relu', name='mlp_mfccs_dense_3')(layer)
    layer = Dense(30, activation='relu', name='mlp_mfccs_dense_4')(layer)
    
    outputs = Dense(25, activation='softmax', name='mlp_mfccs_output')(layer)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=Adam(), 
        loss='categorical_crossentropy', 
        metrics=[
            CategoricalAccuracy(name='accuracy')
        ]
    )
    
    return model

In [6]:
experiment_number = 2

In [9]:
def train_fold(x_train, y_train, x_val, y_val, run_name, fold=None):

    input_shape = x_train[0].shape
    model = create_model(input_shape)
    
    if fold is not None:
        run_name += '_fold' + str(fold)
        print('Training fold ' + str(fold))
    
    tk_board = TensorBoard(log_dir=constants.LOGS_PATH + run_name)
    weight_restorer = BestWeightsRestorer(monitor='val_loss', mode='min', verbose=1)
    
    history = model.fit(
        x=x_train,
        y=y_train,
        validation_data=[x_val, y_val],
        epochs=150,
        batch_size=2000,
        shuffle=True,
        callbacks=[tk_board, weight_restorer],
        verbose=1
    )
    
    prob_predictions = model.predict(x_val)
    predictions = prob_predictions.argmax(axis=1)
    
    return {
        'history': history.history,
        'predictions': predictions
    }

experiment_number += 1
run_name = 'new_base_mlp_mfccs_cv_' + str(experiment_number)

results = resultsUtil.cross_validate(
    x=ftrs,
    y=lbls_oh,
    y_label_encoded=lbl_encoded_lbls,
    n_splits=5,
    train_func=train_fold,
    run_name=run_name
)

runUtil.save_run_results(run_name, results)

resultsdf = pd.DataFrame([results]).transpose()

display(resultsdf)

Training fold 1
Train on 114904 samples, validate on 28740 samples
Epoch 1/150
114904/114904 [==============================] - 1s 4us/sample - loss: 3.1174 - accuracy: 0.0834 - val_loss: 2.9474 - val_accuracy: 0.1275
Epoch 2/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.8725 - accuracy: 0.1421 - val_loss: 2.7607 - val_accuracy: 0.1682
Epoch 3/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.7584 - accuracy: 0.1686 - val_loss: 2.6967 - val_accuracy: 0.1836
Epoch 4/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.7105 - accuracy: 0.1817 - val_loss: 2.6619 - val_accuracy: 0.1945
Epoch 5/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.6778 - accuracy: 0.1902 - val_loss: 2.6342 - val_accuracy: 0.2017
Epoch 6/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.6545 - accuracy: 0.1968 - val_loss: 2.6127 - val_accuracy: 0.2080
Epoch 7/150
114904/114904 [

114904/114904 [==============================] - 0s 2us/sample - loss: 2.5032 - accuracy: 0.2416 - val_loss: 2.4955 - val_accuracy: 0.2397
Epoch 55/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.5033 - accuracy: 0.2414 - val_loss: 2.4961 - val_accuracy: 0.2397
Epoch 56/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.5009 - accuracy: 0.2416 - val_loss: 2.4934 - val_accuracy: 0.2417
Epoch 57/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.5003 - accuracy: 0.2423 - val_loss: 2.4929 - val_accuracy: 0.2405
Epoch 58/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.5011 - accuracy: 0.2416 - val_loss: 2.4933 - val_accuracy: 0.2406
Epoch 59/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.4986 - accuracy: 0.2423 - val_loss: 2.4927 - val_accuracy: 0.2405
Epoch 60/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.4993 - accuracy

Epoch 108/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.4776 - accuracy: 0.2473 - val_loss: 2.4791 - val_accuracy: 0.2465
Epoch 109/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.4767 - accuracy: 0.2463 - val_loss: 2.4797 - val_accuracy: 0.2443
Epoch 110/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.4773 - accuracy: 0.2475 - val_loss: 2.4801 - val_accuracy: 0.2454
Epoch 111/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.4773 - accuracy: 0.2462 - val_loss: 2.4795 - val_accuracy: 0.2443
Epoch 112/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.4749 - accuracy: 0.2481 - val_loss: 2.4787 - val_accuracy: 0.2460
Epoch 113/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.4755 - accuracy: 0.2481 - val_loss: 2.4787 - val_accuracy: 0.2452
Epoch 114/150
114904/114904 [==============================] - 0s 2us/sample - los

Epoch 11/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.5974 - accuracy: 0.2135 - val_loss: 2.5546 - val_accuracy: 0.2263
Epoch 12/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.5880 - accuracy: 0.2153 - val_loss: 2.5486 - val_accuracy: 0.2281
Epoch 13/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.5824 - accuracy: 0.2180 - val_loss: 2.5433 - val_accuracy: 0.2296
Epoch 14/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.5785 - accuracy: 0.2181 - val_loss: 2.5403 - val_accuracy: 0.2309
Epoch 15/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.5740 - accuracy: 0.2199 - val_loss: 2.5358 - val_accuracy: 0.2321
Epoch 16/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.5684 - accuracy: 0.2219 - val_loss: 2.5331 - val_accuracy: 0.2320
Epoch 17/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.56

Epoch 65/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4964 - accuracy: 0.2402 - val_loss: 2.4810 - val_accuracy: 0.2436
Epoch 66/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4947 - accuracy: 0.2415 - val_loss: 2.4813 - val_accuracy: 0.2451
Epoch 67/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4941 - accuracy: 0.2414 - val_loss: 2.4804 - val_accuracy: 0.2454
Epoch 68/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4946 - accuracy: 0.2407 - val_loss: 2.4803 - val_accuracy: 0.2463
Epoch 69/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4932 - accuracy: 0.2422 - val_loss: 2.4803 - val_accuracy: 0.2461
Epoch 70/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4929 - accuracy: 0.2416 - val_loss: 2.4803 - val_accuracy: 0.2464
Epoch 71/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.49

Epoch 119/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4730 - accuracy: 0.2470 - val_loss: 2.4730 - val_accuracy: 0.2464
Epoch 120/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4738 - accuracy: 0.2462 - val_loss: 2.4738 - val_accuracy: 0.2478
Epoch 121/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4732 - accuracy: 0.2480 - val_loss: 2.4740 - val_accuracy: 0.2469
Epoch 122/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4723 - accuracy: 0.2467 - val_loss: 2.4732 - val_accuracy: 0.2470
Epoch 123/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4718 - accuracy: 0.2466 - val_loss: 2.4721 - val_accuracy: 0.2488
Epoch 124/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4737 - accuracy: 0.2458 - val_loss: 2.4722 - val_accuracy: 0.2473
Epoch 125/150
114911/114911 [==============================] - 0s 2us/sample - los

114914/114914 [==============================] - 0s 2us/sample - loss: 2.5471 - accuracy: 0.2282 - val_loss: 2.5209 - val_accuracy: 0.2345
Epoch 23/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.5427 - accuracy: 0.2297 - val_loss: 2.5169 - val_accuracy: 0.2354
Epoch 24/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.5413 - accuracy: 0.2297 - val_loss: 2.5149 - val_accuracy: 0.2341
Epoch 25/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.5384 - accuracy: 0.2304 - val_loss: 2.5130 - val_accuracy: 0.2372
Epoch 26/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.5367 - accuracy: 0.2298 - val_loss: 2.5115 - val_accuracy: 0.2364
Epoch 27/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.5349 - accuracy: 0.2311 - val_loss: 2.5107 - val_accuracy: 0.2372
Epoch 28/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.5352 - accuracy

Epoch 76/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.4911 - accuracy: 0.2421 - val_loss: 2.4821 - val_accuracy: 0.2442
Epoch 77/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.4913 - accuracy: 0.2422 - val_loss: 2.4812 - val_accuracy: 0.2435
Epoch 78/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.4900 - accuracy: 0.2427 - val_loss: 2.4809 - val_accuracy: 0.2438
Epoch 79/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.4911 - accuracy: 0.2421 - val_loss: 2.4821 - val_accuracy: 0.2429
Epoch 80/150
114914/114914 [==============================] - 0s 3us/sample - loss: 2.4897 - accuracy: 0.2438 - val_loss: 2.4805 - val_accuracy: 0.2442
Epoch 81/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.4893 - accuracy: 0.2438 - val_loss: 2.4803 - val_accuracy: 0.2444
Epoch 82/150
114914/114914 [==============================] - 0s 2us/sample - loss: 2.48

KeyboardInterrupt: 

In [14]:
def train_final_model(x, y, run_name):
    
    save_path = constants.MODELS_PATH + run_name + '.h5' 
    
    input_shape = x[0].shape
    model = create_model(input_shape)
    
    tk_board = TensorBoard(log_dir=constants.LOGS_PATH + run_name)
    
    model.fit(
        x=x,
        y=y,
        epochs=150,
        batch_size=2000,
        shuffle=True,
        callbacks=[tk_board],
        verbose=1
    )
    
    model.save(save_path)


train_final_model(ftrs, lbls_oh, 'base_mlp_mfccs_final1')

Epoch 1/150
143644/143644 [==============================] - 1s 4us/sample - loss: 3.0892 - accuracy: 0.1060
Epoch 2/150
143644/143644 [==============================] - 0s 2us/sample - loss: 2.7708 - accuracy: 0.1651
Epoch 3/150
143644/143644 [==============================] - 0s 2us/sample - loss: 2.6729 - accuracy: 0.1913
Epoch 4/150
143644/143644 [==============================] - 0s 2us/sample - loss: 2.6281 - accuracy: 0.2053
Epoch 5/150
143644/143644 [==============================] - 0s 2us/sample - loss: 2.5985 - accuracy: 0.2132
Epoch 6/150
143644/143644 [==============================] - 0s 2us/sample - loss: 2.5774 - accuracy: 0.2198
Epoch 7/150
143644/143644 [==============================] - 0s 2us/sample - loss: 2.5617 - accuracy: 0.2236
Epoch 8/150
143644/143644 [==============================] - 0s 2us/sample - loss: 2.5494 - accuracy: 0.2275
Epoch 9/150
143644/143644 [==============================] - 0s 2us/sample - loss: 2.5393 - accuracy: 0.2299
Epoch 10/150
143644

143644/143644 [==============================] - 0s 2us/sample - loss: 2.3992 - accuracy: 0.2679
Epoch 150/150
143644/143644 [==============================] - 0s 2us/sample - loss: 2.3990 - accuracy: 0.2670
